# Training cycle debugging

Basically run code copied from train_seq2seq.py in this notebook to catch bugs

In [1]:
import os
os.environ['ALFRED_ROOT'] = '/home/hoyeung/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

# from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
import torch
import pprint
import json
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from models.utils.helper_utils import optimizer_to

In [2]:
# import these if we want to debug a model such as seq2seq_nl_baseline.py

import torch
import pprint
import json
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from models.utils.helper_utils import optimizer_to

# Parser

In [3]:
# These are the default flags present in train_seq2seq.py

parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# settings
args.seed = 123
args.data = 'data/json_feat_2.1.0'
args.splits = 'data/splits/oct21.json'
args.preprocess = False #!
args.pp_folder = 'pp'
args.monitor_train_every = 10
args.save_every_epoch = False #!
args.model = 'seq2seq_nl_baseline'
args.gpu = True
args.dout = 'exp/model:seq2seq_nl_baseline'
args.resume = False #!

# hyper parameters
args.batch = 8
args.epoch = 20
args.lr = 1e-4
args.decay_epoch = 10
args.dhid = 512
args.dframe = 2500
args.demb = 100
args.pframe = 300
args.mask_loss_wt = 1.
args.action_loss_wt = 1.
args.subgoal_aux_loss_wt = 0.
args.pm_aux_loss_wt = 0.

# dropouts
args.zero_goal = False #!
args.zero_instr = False #!
args.act_dropout = 0.
args.lang_dropout = 0.
args.input_dropout = 0.
args.vis_dropout = 0.3
args.hstate_dropout = 0.3
args.attn_dropout = 0.
args.actor_dropout = 0.
args.word_dropout = 0.

# other settings
args.dec_teacher_forcing = False #!
args.temp_no_history = False #!

# debugging
args.fast_epoch = False #!
args.dataset_fraction = 0

In [4]:
# overwrite the default flags

args.preprocess = False # Turn this to True if running for the first time

args.model = 'seq2seq_nl_baseline'  # found under models/model/ directory
args.dout = 'exp/model:seq2seq_nl_baseline'

args.dec_teacher_forcing = True
args.gpu = False

# light setup for debugging
args.fast_epoch = True # Turn this to False if running for the first time to preprocess data properly
args.epoch = 20

In [5]:
torch.manual_seed(args.seed)

# Setup and load data splits

In [6]:
# make output dir
if not os.path.isdir(args.dout):
    os.makedirs(args.dout)

print('args.out ', args.dout)

args.out  exp/model:seq2seq_nl_baseline


In [7]:
# load train/valid/tests splits
with open(args.splits) as f:
    splits = json.load(f)
    pprint.pprint({k: len(v) for k, v in splits.items()})

{'tests_seen': 1533,
 'tests_unseen': 1529,
 'train': 21023,
 'valid_seen': 820,
 'valid_unseen': 821}


In [8]:
# preprocess and save -- only need to preprocess once
if args.preprocess:
    print("\nPreprocessing dataset and saving to %s folders ... This will take a while. Do this once as required." % args.pp_folder)
    dataset = Dataset(args, None)
    dataset.preprocess_splits(splits)
    vocab = torch.load(os.path.join(args.dout, "%s.vocab" % args.pp_folder))
else:
    vocab = torch.load(os.path.join(args.data, "%s.vocab" % args.pp_folder))
    
print(vocab)

{'word': Vocab(2360), 'action_low': Vocab(15), 'action_high': Vocab(93)}


# Model and Training

In [9]:
# load the model architecture
args.gpu = False
args.resume = None # os.path.join(args.dout, 'best_seen.pth')

M = import_module('model.{}'.format(args.model))
if args.resume:
    print("Loading: " + args.resume)
    model, optimizer, start_epoch = M.Module.load(args.resume)
    print("Restarting at epoch {}/{}".format(start_epoch, args.epoch-1))
    if start_epoch >= args.epoch:
        print('Checkpoint already finished {}/{} epochs.'.format(start_epoch, args.epoch))
        sys.exit(0)
else:
    model = M.Module(args, vocab)
    optimizer = None
    start_epoch = 0

if args.gpu:
    model = model.to(torch.device('cuda'))
    if not optimizer is None:
        optimizer_to(optimizer, torch.device('cuda'))

In [10]:
# examine model layers
model

Module(
  (emb_word): Embedding(2360, 100)
  (emb_action_low): Embedding(15, 100)
  (enc): LSTM(100, 512, batch_first=True, bidirectional=True)
  (enc_att): SelfAttn(
    (scorer): Linear(in_features=1024, out_features=1, bias=True)
  )
  (dec): LanguageDecoder(
    (emb): Embedding(2360, 100)
    (cell): LSTMCell(1124, 1024)
    (attn): DotAttn()
    (input_dropout): Dropout(p=0.0, inplace=False)
    (attn_dropout): Dropout(p=0.0, inplace=False)
    (hstate_dropout): Dropout(p=0.3, inplace=False)
    (word_dropout): Dropout(p=0.0, inplace=False)
    (word): Linear(in_features=2148, out_features=100, bias=True)
    (h_tm1_fc): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (act_dropout): Dropout(p=0.0, inplace=True)
)

In [11]:
# main training loop -- debug here if breakpoints were inserted
model.run_train(splits, optimizer=optimizer, start_epoch=start_epoch)

batch:   0%|          | 0/2 [00:00<?, ?it/s]

Saving to: exp/model:seq2seq_nl_baseline



batch: 100%|██████████| 2/2 [00:10<00:00,  5.35s/it]

batch: 100%|██████████| 2/2 [00:06<00:00,  3.21s/it]
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/nltk/translate/bleu_score.py:523: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score ev


Found new best valid_seen!! Saving...
Found new best valid_unseen!! Saving...


batch:   0%|          | 0/2 [00:00<?, ?it/s]

...

{'epoch': 18,
 'train': {'loss_lang_instr': 3.1863350868225098,
           'total_loss': 3.1863350868225098},
 'valid_seen': {'BLEU': 0.006230817273158473,
                'loss_lang_instr': 4.167171001434326,
                'total_loss': 4.167171001434326},
 'valid_unseen': {'BLEU': 0.002381648093789713,
                  'loss_lang_instr': 3.522326707839966,
                  'total_loss': 3.522326707839966}}
epoch_time                    279.445                                 
forward_batch_train           131.527                                 
forward_batch_valid_seen      54.948                                  
forward_batch_valid_unseen    52.916                                  
torch_save_valid_seen         9.36                                    
compute_metrics_valid_seen    8.983                                   
compute_metrics_valid_unseen  8.373                                   
torch_save_last               6.194                                   
torch_save_valid


batch: 100%|██████████| 2/2 [00:09<00:00,  4.64s/it]

batch: 100%|██████████| 2/2 [00:02<00:00,  1.26s/it]

batch: 100%|██████████| 2/2 [00:02<00:00,  1.15s/it]



Found new best valid_seen!! Saving...


epoch: 100%|██████████| 20/20 [04:57<00:00, 14.89s/it]

{'epoch': 19,
 'train': {'loss_lang_instr': 3.1608837842941284,
           'total_loss': 3.1608837842941284},
 'valid_seen': {'BLEU': 0.006351188978705782,
                'loss_lang_instr': 4.219351053237915,
                'total_loss': 4.219351053237915},
 'valid_unseen': {'BLEU': 0.001968472204978389,
                  'loss_lang_instr': 3.4981753826141357,
                  'total_loss': 3.4981753826141357}}
epoch_time                    297.739                                 
forward_batch_train           140.8                                   
forward_batch_valid_seen      57.485                                  
forward_batch_valid_unseen    55.246                                  
torch_save_valid_seen         11.96                                   
compute_metrics_valid_seen    9.276                                   
compute_metrics_valid_unseen  8.696                                   
torch_save_last               6.95                                    
torch_save_val